In [13]:
lib = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/'
import sys
sys.path.append(lib)
from utilities.package_importing import *


# Finding a threshold for variance so that we will have 4k~ genes.

##### Loads pickle file of NORMALIZED cohort

In [2]:
# cohort_path = r'/storage/md_keren/shitay/Data/droplet_seq/cohort/normalized/6.21/cohort_normalized_26.6.21.pkl'
# cohort_path = r'/storage/md_keren/shitay/Data/droplet_seq/cohort/normalized/6.21/myeloid_normalized_26.6.21.pkl'
# cohort_path = fr'/storage/md_keren/shitay/Data/droplet_seq/M97_M173/cohort/normalized/4.11.21/cohort_normalized_4.11.21.pkl'
cohort_path = r'/storage/md_keren/shitay/Data/droplet_seq/M97_M173/subcohort/normalized/1.1.22/sub_cohort_normalized_1.1.22_protein_coding_genes.pkl'

normalized_cohort = pickle.load(open(cohort_path, 'rb'))

##### Run the next block if you want to keep only protein coding genes:

In [3]:
print(f'Number of genes in cohort: {normalized_cohort.number_of_genes}')
normalized_cohort.filter_protein_coding_genes()
print(f'Number of genes in cohort after filtering: {normalized_cohort.number_of_genes}')

Number of genes in cohort: 32738
Dataset was cleared from non-protein coding genes
Number of genes in cohort after filtering: 20273


In [4]:
out_path = r'/storage/md_keren/shitay/Data/droplet_seq/M97_M173/cohort/normalized/4.11.21/cohort_normalized_4.11.21_protein_coding_genes.pkl'
print(f'Number of genes in file: {normalized_cohort.number_of_genes}')
pickle.dump((normalized_cohort), open(out_path, 'wb'), protocol=4)
print('Done')

Number of genes in file: 20273
Done


##### Finds a threshold for all cohort

In [43]:
variances = np.var(normalized_cohort.counts, axis=0)
variances_sorted = np.flip(variances[np.argsort(variances)])

In [44]:
num_requested_genes = 4000
print('The 20 variances around the 4Kth gene:')
print(variances_sorted[num_requested_genes-10: num_requested_genes+10], end='\n\n')
print(f'To get 4K~ genes, you need to take a number between {variances_sorted[num_requested_genes-1: num_requested_genes+1]}')
all_cohort_variance_threshold = variances_sorted[num_requested_genes-1]+ (variances_sorted[num_requested_genes]-variances_sorted[num_requested_genes-1])/2
print(f'You can take: {all_cohort_variance_threshold}')


The 20 variances around the 4Kth gene:
[0.28921257 0.28910463 0.28902773 0.28886497 0.28879879 0.28871017
 0.28868781 0.28868044 0.2886425  0.28861065 0.28853778 0.28846126
 0.28845333 0.28828159 0.2881256  0.2879871  0.28792872 0.28781682
 0.28773422 0.28765617]

To get 4K~ genes, you need to take a number between [0.28861065 0.28853778]
You can take: 0.28857421811408934


##### Finds a threshold for immune cells

In [14]:
immune_cells = normalized_cohort.filter_cells_by_property('is_immune', True)

In [16]:
variances = np.var(immune_cells.counts, axis=0)
variances_sorted = np.flip(variances[np.argsort(variances)])

In [17]:
num_requested_genes = 4000
print('The 20 variances around the 4Kth gene:')
print(variances_sorted[num_requested_genes-10: num_requested_genes+10], end='\n\n')
print(f'To get 4K~ genes, you need to take a number between {variances_sorted[num_requested_genes-1: num_requested_genes+1]}')
immune_optimal_threshold = variances_sorted[num_requested_genes-1]+ (variances_sorted[num_requested_genes]-variances_sorted[num_requested_genes-1])/2
print(f'You can take: {immune_optimal_threshold}')

The 20 variances around the 4Kth gene:
[0.32477845 0.32468761 0.324658   0.32451938 0.32409917 0.32384286
 0.32360099 0.32353323 0.32350215 0.32349147 0.32344303 0.32343794
 0.32343003 0.32338269 0.32338191 0.3233039  0.32322295 0.32319925
 0.32308194 0.32303047]

To get 4K~ genes, you need to take a number between [0.32349147 0.32344303]
You can take: 0.3234672507459016


##### Finds a threshold for tumor cells

In [5]:
tumor_cells = normalized_cohort.filter_cells_by_property('is_cancer', True)


In [6]:
variances = np.var(tumor_cells.counts, axis=0)
variances_sorted = np.flip(variances[np.argsort(variances)])

In [8]:
num_requested_genes = 4000
print('The 20 variances around the 4Kth gene:')
print(variances_sorted[num_requested_genes-10: num_requested_genes+10], end='\n\n')
print(f'To get 4K~ genes, you need to take a number between {variances_sorted[num_requested_genes-1: num_requested_genes+1]}')
print(f'You can take: {variances_sorted[num_requested_genes-1]+ (variances_sorted[num_requested_genes]-variances_sorted[num_requested_genes-1])/2}')

The 20 variances around the 4Kth gene:
[0.27668357 0.27666568 0.27664868 0.27649206 0.27641217 0.27637867
 0.27637441 0.27628617 0.27619825 0.27618969 0.27613441 0.27611972
 0.27597427 0.27596926 0.27580344 0.27543347 0.27541456 0.27532683
 0.2753113  0.27526536]

To get 4K~ genes, you need to take a number between [0.27618969 0.27613441]
You can take: 0.276162053656859


### Saves PKLs of immune, tumor, or all cells with after variance filter

Tumor:

In [10]:
out_path = r'/storage/md_keren/shitay/Data/droplet_seq/M97_M173/cohort/normalized/4.11.21/tumor_cells_normalized_4.11.21_4k_genes.pkl'
variance = 0.276162053656859 # [0.27510745 0.27507093]

tumor_filtered = tumor_cells.filter_genes_by_variance(variance)
print(f'Number of genes in file: {tumor_filtered.number_of_genes}')
pickle.dump((tumor_filtered), open(out_path, 'wb'), protocol=4)
print('Done')

Dataset was cleared from genes with variance of less than 0.276162053656859
Number of genes in file: 4000
Done


Immune:

In [8]:
# out_path = r'/storage/md_keren/shitay/Data/droplet_seq/cohort/normalized/6.21/immune_cells_26.6.21_4k_genes.pkl'
out_path = r'/storage/md_keren/shitay/Data/droplet_seq/M97_M173/subcohort/normalized/1.1.22/immune_cells_normalized_1.1.22_4k_genes.pkl'
variance = immune_optimal_threshold # 0.31975473431525625

immune_filtered = immune_cells.filter_genes_by_variance(variance)
print(f'Number of genes in file: {immune_filtered.number_of_genes}')
pickle.dump((immune_filtered), open(out_path, 'wb'), protocol=4)
print('Done')

Dataset was cleared from genes with variance of less than 0.3234672507459016
Number of genes in file: 4000
Done


All cells (cohort):

In [45]:
out_path = r'/storage/md_keren/shitay/Data/droplet_seq/cohort/normalized/6.21/myeloid_normalized_26.6.21_4k_genes.pkl'
variance = all_cohort_variance_threshold

cohort_filtered = normalized_cohort.filter_genes_by_variance(variance)
print(f'Number of genes in file: {cohort_filtered.number_of_genes}')
pickle.dump((cohort_filtered), open(out_path, 'wb'), protocol=4)
print('Done')

Dataset was cleared from genes with variance of less than 0.28857421811408934
Number of genes in file: 4000
Done


# Protein coding overlap with high variance genes (4K first genes)

Loads protein coding DF:

In [30]:
PROTEIN_CODING_FILE = r'/storage/md_keren/shitay/Data/tables/gene_ens_map.xlsx'
protein_coding_df = pandas.read_excel(PROTEIN_CODING_FILE, header = None, names=['gene_id', 'gene', '1', '2'])
protein_coding_genes = protein_coding_df[protein_coding_df['2'] == 'protein_coding']['gene'].tolist()

Loads cohort that you want to see the number of protein coding genes out of all genes in the cohort:

In [31]:
cohort_path = r'/storage/md_keren/shitay/Data/droplet_seq/cohort/normalized/6.21/cytotoxic_normalized_26.6.21_4k_genes.pkl'
normalized_cohort = pickle.load(open(cohort_path, 'rb'))

In [32]:
overlap_genes = intersection_of_lists(normalized_cohort.gene_names, protein_coding_genes)
print(f'Number of protein coding genes if DF: {len(protein_coding_genes)}')
print(f'Number of genes in cohort: {normalized_cohort.number_of_genes}')
print(f'Number of protein coding genes in cohort: {len(overlap_genes)}')

Number of protein coding genes if DF: 20776
Number of genes in cohort: 4000
Number of protein coding genes in cohort: 4000


# Debug

In [34]:
normalized_cohort.number_of_cells

25815

In [39]:
tumor_cells.number_of_cells
sum(tumor_cells.cells_information.getattr('is_cancer'))

97225

In [44]:
ii = pickle.load(open(r'/storage/md_keren/shitay/Data/droplet_seq/cohort/normalized/5.21/immune_cells_4k_genes.pkl', 'rb'))

In [56]:
ii.counts[700:900].max()

10.936111592118117

In [77]:
sum(tumor_filtered.cells_information.getattr('is_cancer'))
tumor_filtered.number_of_cells

97225

In [62]:
cohort_path = r'/storage/md_keren/shitay/Data/droplet_seq/cohort/normalized/5.21/cohort_normalized_10.5.21.pkl'
normalized_cohort = pickle.load(open(cohort_path, 'rb'))

In [66]:
save_path = r'/storage/md_keren/shitay/Data/droplet_seq/cohort/normalized/5.21/immune_cells_10.5.21.pkl'
immune_cells = normalized_cohort.filter_cells_by_property('is_immune', True)

In [70]:
immune_cells.number_of_genes
pickle.dump((immune_cells), open(save_path, 'wb'), protocol=4)

In [7]:
tt = pickle.load(open(out_path, 'rb'))
tt


In [9]:
tt.number_of_genes

4000

In [4]:
cohort_path = r'/storage/md_keren/shitay/Data/droplet_seq/M97_M173/subcohort/normalized/1.1.22/subcohort_normalized_1.1.22_protein_coding_genes.pkl'

normalized_cohort = pickle.load(open(cohort_path, 'rb'))

In [7]:

MYELOID_CLUSTER_IDX = 5

cohort = normalized_cohort
SUBSET = 'MYELOIDS'
# ANALYSIS PARAMS for MYELOIDS, taken from summary of immune clustering analysis - cohort 26.6.21
# IMMUNE_CLUSTERING_PATH = r'/storage/md_keren/shitay/outputs/clustering/immune/summaries/26.6.21/immune_kmeans_26.6.21_clusters_mapping.csv'
# new clustering mapping  path (integrated data M97-M173)
IMMUNE_CLUSTERING_PATH = r'/storage/md_keren/shitay/outputs/clustering/immune/summaries/4.11.21/immune_kmeans_integrated_4.11.21_clusters_mapping.csv'

if SUBSET is None:
    print(f'No subset request was given')
    res =  cohort
elif SUBSET == 'MYELOIDS':
    print(f'Will be performed on myeloids')
    print(f'IMMUNE CLUSTERING in path \t {IMMUNE_CLUSTERING_PATH} \t is used for defining which cells are myeloids')
    print(f'Cluster {MYELOID_CLUSTER_IDX} is defined to be myeloids')

    immune_mapping = pd.read_csv(IMMUNE_CLUSTERING_PATH)[['Sample', 'Barcode', 'Cluster']]
    myeloid_indices = immune_mapping[immune_mapping['Cluster'] == MYELOID_CLUSTER_IDX][['Sample', 'Barcode']].values

    res =  cohort.get_subset_by_identifiers(myeloid_indices[:, 0], myeloid_indices[:, 1])

Will be performed on myeloids
IMMUNE CLUSTERING in path 	 /storage/md_keren/shitay/outputs/clustering/immune/summaries/4.11.21/immune_kmeans_integrated_4.11.21_clusters_mapping.csv 	 is used for defining which cells are myeloids
Cluster 5 is defined to be myeloids


In [10]:
res.counts.shape

(7645, 20273)